In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms import v2
from torchvision.transforms.v2 import functional as vF
from torchvision.ops import box_convert
from einops import rearrange
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

# Dataset and DataLoader

In [ ]:
# from datasets import load_dataset
# train_dataset = load_dataset(import os
import torch
import torchvision.transforms.v2
from torch.utils.data.dataset import Dataset
import xml.etree.ElementTree as ET
from torchvision import tv_tensors
from torchvision.io import read_image


def load_images_and_anns(im_sets, label2idx, ann_fname, split):
    r"""
    Method to get the xml files and for each file
    get all the objects and their ground truth detection
    information for the dataset
    :param im_sets: Sets of images to consider
    :param label2idx: Class Name to index mapping for dataset
    :param ann_fname: txt file containing image names{trainval.txt/test.txt}
    :param split: train/test
    :return:
    """
    im_infos = []
    for im_set in im_sets:
        im_names = []
        # Fetch all image names in txt file for this imageset
        for line in open(os.path.join(
                im_set, 'ImageSets', 'Main', '{}.txt'.format(ann_fname))):
            im_names.append(line.strip())

        # Set annotation and image path
        ann_dir = os.path.join(im_set, 'Annotations')
        im_dir = os.path.join(im_set, 'JPEGImages')

        for im_name in im_names:
            ann_file = os.path.join(ann_dir, '{}.xml'.format(im_name))
            im_info = {}
            ann_info = ET.parse(ann_file)
            root = ann_info.getroot()
            size = root.find('size')
            width = int(size.find('width').text)
            height = int(size.find('height').text)
            im_info['img_id'] = os.path.basename(ann_file).split('.xml')[0]
            im_info['filename'] = os.path.join(
                im_dir, '{}.jpg'.format(im_info['img_id'])
            )
            im_info['width'] = width
            im_info['height'] = height
            detections = []
            for obj in ann_info.findall('object'):
                det = {}
                label = label2idx[obj.find('name').text]
                difficult = int(obj.find('difficult').text)
                bbox_info = obj.find('bndbox')
                bbox = [
                    int(bbox_info.find('xmin').text) - 1,
                    int(bbox_info.find('ymin').text) - 1,
                    int(bbox_info.find('xmax').text) - 1,
                    int(bbox_info.find('ymax').text) - 1
                ]
                det['label'] = label
                det['bbox'] = bbox
                det['difficult'] = difficult
                detections.append(det)
            im_info['detections'] = detections
            # Because we are using 25 as num_queries,
            # so we ignore all images in VOC with greater
            # than 25 target objects.
            # This is okay, since this just means we are
            # ignoring a small number of images(15 to be precise)
            if len(detections) <= 25:
                im_infos.append(im_info)
    print('Total {} images found'.format(len(im_infos)))
    return im_infos


class VOCDataset(Dataset):
    def __init__(self, split, im_sets, im_size=640):
        self.split = split

        # Imagesets for this dataset instance (VOC2007/VOC2007+VOC2012/VOC2007-test)
        self.im_sets = im_sets
        self.fname = 'trainval' if self.split == 'train' else 'test'
        self.im_size = im_size
        self.im_mean = [123.0, 117.0, 104.0]
        self.imagenet_mean = [0.485, 0.456, 0.406]
        self.imagenet_std = [0.229, 0.224, 0.225]

        # Train and test transformations
        self.transforms = {
            'train': torchvision.transforms.v2.Compose([
                torchvision.transforms.v2.RandomHorizontalFlip(p=0.5),
                torchvision.transforms.v2.RandomZoomOut(fill=self.im_mean),
                torchvision.transforms.v2.RandomIoUCrop(),
                torchvision.transforms.v2.RandomPhotometricDistort(),
                torchvision.transforms.v2.Resize(size=(self.im_size, self.im_size)),
                torchvision.transforms.v2.SanitizeBoundingBoxes(
                    labels_getter=lambda transform_input:
                    (transform_input[1]["labels"], transform_input[1]["difficult"])),
                torchvision.transforms.v2.ToPureTensor(),
                torchvision.transforms.v2.ToDtype(torch.float32, scale=True),
                # torchvision.transforms.v2.Normalize(mean=self.imagenet_mean, std=self.imagenet_std)

            ]),
            'test': torchvision.transforms.v2.Compose([
                torchvision.transforms.v2.Resize(size=(self.im_size, self.im_size)),
                torchvision.transforms.v2.ToPureTensor(),
                torchvision.transforms.v2.ToDtype(torch.float32, scale=True),
                # torchvision.transforms.v2.Normalize(mean=self.imagenet_mean, std=self.imagenet_std)
            ]),
        }

        classes = [
            'person', 'bird', 'cat', 'cow', 'dog', 'horse', 'sheep',
            'aeroplane', 'bicycle', 'boat', 'bus', 'car', 'motorbike', 'train',
            'bottle', 'chair', 'diningtable', 'pottedplant', 'sofa', 'tvmonitor'
        ]
        classes = sorted(classes)
        # We need to add background class as well with 0 index
        classes = ['background'] + classes

        self.label2idx = {classes[idx]: idx for idx in range(len(classes))}
        self.idx2label = {idx: classes[idx] for idx in range(len(classes))}
        print(self.idx2label)
        self.images_info = load_images_and_anns(self.im_sets,
                                                self.label2idx,
                                                self.fname,
                                                self.split)

    def __len__(self):
        return len(self.images_info)

    def __getitem__(self, index):
        im_info = self.images_info[index]
        im = read_image(im_info['filename'])
        # print(im)

        # Get annotations for this image
        targets = {}
        targets['boxes'] = tv_tensors.BoundingBoxes(
            [detection['bbox'] for detection in im_info['detections']],
            format='XYXY', canvas_size=im.shape[-2:])
        targets['labels'] = torch.as_tensor(
            [detection['label'] for detection in im_info['detections']])
        targets['difficult'] = torch.as_tensor(
            [detection['difficult']for detection in im_info['detections']])

        # Transform the image and targets
        transformed_info = self.transforms["test"](im, targets)
        im_tensor, targets = transformed_info
        # print(im_tensor)

        h, w = im_tensor.shape[-2:]

        # Boxes returned are in x1y1x2y2 format normalized from 0-1
        wh_tensor = torch.as_tensor([[w, h, w, h]]).expand_as(targets['boxes'])
        # return im_tensor, targets, im_info['filename']
        return_dict = {
            'img': im_tensor,
            'bboxes': targets['boxes'] / 320,
            'cls': targets['labels'],
            # 'segments':,
            # 'keypoints':,
            # 'mask':,
            'batch_idx': torch.zeros(targets['labels'].shape[0], dtype=torch.int64),
            'bbox_format': 'xyxy',
            'normalized': True,
        }
        return return_dict
        # {
        #   "img":        Tensor[C,H,W],           # the image tensor
        #   "bboxes":     Tensor[N,4],             # N boxes as float32, in whatever format (XYXY or XYWH)
        #   "cls":        Tensor[N,1] or Tensor[N],# N class indices
        #   "segments":   List[np.ndarray] or [],  # empty list if no segments
        #   "keypoints":  List[...] or None,       # None if no keypoints
        #   "mask":       Tensor[...] (optional),  # if doing segmentation masks
        #   "batch_idx":  Tensor[N],               # zeros (or the image’s batch index) per box
        #   "bbox_format": str,                    # e.g. "xywh" or "xyxy"
        #   "normalized": bool                     # True if box coords are in [0,1]
        # }

# test_dataset = load_dataset("detection-datasets/coco", split='val', streaming=True)

In [ ]:
transforms = v2.Compose([
    # v2.RGB(),
    # v2.ToImage(),  # Convert to tensor, only needed if you had a PIL image
    # v2.PILToTensor(),
    # v2.ToDtype(torch.uint8, scale=True),  # optional, most input are already uint8 at this point
    # ...
    # v2.Resize(size=(320, 320), antialias=True),  # Or Resize(antialias=True)
    # ...
    v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    # v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
from torchvision import tv_tensors
def preprocess_ds(batch):
    img = batch[0]
    w = img.shape[1]
    h = img.shape[2]
    obj = batch[1]
    boxes_xyxy = obj['boxes'].to(torch.float32)
    output = {}
    output['new_image'] = img
    output['boxes_xyxy'] = tv_tensors.BoundingBoxes(
        boxes_xyxy,
        format="XYXY", canvas_size=(1, 1)
    )
    output['labels'] = obj["labels"]
    obj_mask = torch.zeros(320, 320)
    for obx in boxes_xyxy:
        x1, y1, x2, y2 = tuple(torch.round(obx * 320).to(torch.int32))
        # print(x1, y1, x2, y2)
        obj_mask[x1:x2, y1:y2] = 1
    output['mask'] = obj_mask
    return output

In [ ]:
%%time
def collate_function(data):
    return tuple(zip(*data))


voc = VOCDataset('train',
                 im_sets=["/kaggle/input/pascal-voc-2007/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007"],
                 im_size=320)
# train_ds = []
# for i in tqdm(range(len(voc))):
#     train_ds.append(preprocess_ds(voc[i]))
voc2 = VOCDataset('test',
                 im_sets=["/kaggle/input/pascal-voc-2007/VOCtest_06-Nov-2007/VOCdevkit/VOC2007"],
                 im_size=320)
# test_ds = []
# for i in tqdm(range(len(voc2))):
#     test_ds.append(preprocess_ds(voc2[i]))

In [ ]:
# voc[0]

In [ ]:
# train_augs = v2.Compose([
#     v2.RandomRotation(30),
#     v2.RandomHorizontalFlip(p=0.5),
#     v2.ClampBoundingBoxes(),
# ])

In [ ]:
# import random

# def collate_fn(in_batch):
#     images = []
#     targets = []
#     for batch in in_batch:
#         img = batch['new_image']
#         lbl = batch['labels']
#         bbox = batch['boxes_xyxy']
#         mask = batch['mask']
#         img, bbox = train_augs(img, bbox)
#         # images.append(img)
#         temp = {
#             "img": img.unsqueeze(0), # 4D Batched input
#             "labels": lbl,
#             "boxes": bbox,
#             # "mask": mask
#         }
#         targets.append(temp)
#     return targets

In [ ]:
from ultralytics.data.dataset import YOLOConcatDataset

In [ ]:
train_loader = DataLoader(
    voc,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    collate_fn=YOLOConcatDataset.collate_fn,
)
test_loader = DataLoader(
    voc2,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    collate_fn=YOLOConcatDataset.collate_fn,
)

In [ ]:
%%time
for batch in train_loader:
    # print(batch)
    new_batch = {}
    new_batch['batch_idx'] = batch['batch_idx'].to(device)
    new_batch['bboxes'] = batch['bboxes'].to(device)
    new_batch['bbox_format'] = batch['bbox_format']
    new_batch['cls'] = batch['cls'].to(device)
    new_batch['img'] = batch['img'].to(device)
    new_batch['normalized'] = batch['normalized']
    break

# Pretrained YOLO Model

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolo11m.pt').to(device)
model = model.model # VERY IMPORTANT, AS THE OUTER MODEL IS WRAPPED IN YOLO CLASS

In [ ]:
from types import SimpleNamespace
model.args = SimpleNamespace(box=7.5, cls=0.5, dfl=1.5, pose=12.0, kobj=1.0)

In [ ]:
sum(p.numel() for p in model.parameters())

In [ ]:
# model

In [ ]:
help(model.forward)

In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:
%%time
model.train()
for batch in train_loader:
    new_batch = {}
    new_batch['batch_idx'] = batch['batch_idx'].to(device)
    new_batch['bboxes'] = batch['bboxes'].to(device)
    new_batch['bbox_format'] = batch['bbox_format']
    new_batch['cls'] = batch['cls'].to(device)
    new_batch['img'] = batch['img'].to(device)
    new_batch['normalized'] = batch['normalized']
    loss_components, others = model(new_batch)
    print(loss_components)
    print(others)
    loss = loss_components.mean()
    loss.backward()
    break

# Train Loop

In [ ]:
from tqdm import tqdm

In [ ]:
def train():
    model.train()
    total_loss = 0
    cnt = 0
    for batch in (pbar := tqdm(train_loader)):
        new_batch = {}
        new_batch['batch_idx'] = batch['batch_idx'].to(device)
        new_batch['bboxes'] = batch['bboxes'].to(device)
        new_batch['bbox_format'] = batch['bbox_format']
        new_batch['cls'] = batch['cls'].to(device)
        new_batch['img'] = batch['img'].to(device)
        new_batch['normalized'] = batch['normalized']
        loss_components, _ = model(new_batch)
        loss = loss_components.mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        cnt += 1
        pbar.set_description(f"Average Loss: {total_loss / cnt :6f}")

In [ ]:
def test():
    model.eval()
    total_loss = 0
    cnt = 0
    with torch.no_grad():
        for batch in (pbar := tqdm(test_loader)):
            new_batch = {}
            new_batch['batch_idx'] = batch['batch_idx'].to(device)
            new_batch['bboxes'] = batch['bboxes'].to(device)
            new_batch['bbox_format'] = batch['bbox_format']
            new_batch['cls'] = batch['cls'].to(device)
            new_batch['img'] = batch['img'].to(device)
            new_batch['normalized'] = batch['normalized']
            loss_components, _ = model(new_batch)
            loss = loss_components.mean()
            total_loss += loss.item()
            cnt += 1
            pbar.set_description(f"Testing: Average Loss: {total_loss / cnt :6f}")

# Actual Training

In [ ]:
model = YOLO('yolo11m.pt').to(device)
model = model.model # VERY IMPORTANT, AS THE OUTER MODEL IS WRAPPED IN YOLO CLASS

In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:
from types import SimpleNamespace
model.args = SimpleNamespace(box=7.5, cls=0.5, dfl=1.5, pose=12.0, kobj=1.0)

In [ ]:
sum(p.numel() for p in model.parameters())

In [ ]:
from torch.optim import *

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-4)

In [ ]:
train()

In [ ]:
test()

# Visualisation

In [ ]:
inference_model = YOLO('yolo11m.pt').to(device)

In [ ]:
inference_model.model.load_state_dict(model.state_dict())

In [ ]:
batch['img'].shape

In [ ]:
inference_model.eval()
with torch.no_grad():
    for batch in (pbar := tqdm(test_loader)):
        results = inference_model.predict(              # run batched inference + NMS
            batch['img'],                             # list/array/tensor of images
            conf=0.75,                        # confidence threshold
            iou=0.45,                         # NMS IoU threshold
            device=device,                   # 'cpu' or 'cuda'
            stream=False                      # returns list of Results
        )
        
        # unpack the first image’s detections:
        res     = results[0]
        boxes   = res.boxes.xyxy.cpu().numpy()   # (N,4) array of x1,y1,x2,y2
        scores  = res.boxes.conf.cpu().numpy()   # (N,) objectness scores
        classes = res.boxes.cls.cpu().numpy()    # (N,) zero-based class IDs        
        break

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(batch['img'][1].permute(1, 2, 0).cpu().numpy())